# Model Fine Tuning

# Notebook Environment

For a unified research environment, enable the flags below:

In [ ]:
UPGRADE_PY = False
INSTALL_DEPS = False
if INSTALL_DEPS:
  # !pip install -q tensorboard==2.15.2
  # !pip install -q tensorflow[and-cuda]==2.15.1
  # !pip install -q tensorflow==2.15.0
  # !pip install -q tensorflow-io-gcs-filesystem==0.36.0
  # !pip install -q tensorflow-text==2.15.0
  # !pip install -q tf_keras==2.15.1
  # !pip install -q tokenizers==0.15.2
  # !pip install -q torch==2.2.0+cpu
  # !pip install -q torch-xla==2.2.0+libtpu
  # !pip install -q torchdata==0.7.1
  !pip install -q transformers==4.38.2

if UPGRADE_PY:
    !mamba create -n py311 -y
    !source /opt/conda/bin/activate py312 && mamba install python=3.11 jupyter mamba -y

    !sudo rm /opt/conda/bin/python3
    !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3
    !sudo rm /opt/conda/bin/python3.10
    !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python3.10
    !sudo rm /opt/conda/bin/python
    !sudo ln -sf /opt/conda/envs/py312/bin/python3 /opt/conda/bin/python

!python --version

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

# Transformers cannot use keras3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_USE_LEGACY_KERAS'] = '1'
IN_KAGGLE = IN_COLAB = False
!export CUDA_LAUNCH_BLOCKING=1
!export XLA_FLAGS=--xla_cpu_verbose=0

try:
    # https://www.tensorflow.org/install/pip#windows-wsl2
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = "/content/drive/MyDrive/EDT dataset"
    MODEL_PATH = "/content/drive/MyDrive/models"
    IN_COLAB = True
    print('Colab!')
except:
    IN_COLAB = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ and not IN_COLAB:
    print('Running in Kaggle...')
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    MODEL_PATH = "./models"
    DATA_PATH = "/kaggle/input"
    IN_KAGGLE = True
    print('Kaggle!')
elif not IN_COLAB and not IN_KAGGLE:
    IN_KAGGLE = False
    MODEL_PATH = "./models"
    DATA_PATH = "./data"
    print('Normal!')

MODEL_BASE = "google-bert/bert-base-cased"

# Accelerators Configuration

If you have a GPU, TPU or in one of the collaborative notebooks. Configure your setup below:

In [ ]:
import numpy as np
import math
import shutil
import pandas as pd

from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import mixed_precision

print(f'Tensorflow version: [{tf.__version__}]')

tf.get_logger().setLevel('INFO')

#tf.config.set_soft_device_placement(True)
#tf.config.experimental.enable_op_determinism()
#tf.random.set_seed(1)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except Exception as e:
    # Not an exception, just no TPUs available, GPU is fallback
    # https://www.tensorflow.org/guide/mixed_precision
    print(e)
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if len(gpus) > 0:

        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, False)
            tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12288)])
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            strategy = tf.distribute.MirroredStrategy()

            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
        finally:
            print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
    else:
        # CPU is final fallback
        strategy = tf.distribute.get_strategy()
        print("Running on CPU")

def is_tpu_strategy(strategy):
    return isinstance(strategy, tf.distribute.TPUStrategy)

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

# Fine-Tuning with Masked Models

In [ ]:
from transformers import BertTokenizerFast,TFBertForMaskedLM

# https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#berttokenizerfast
tokenizer = BertTokenizerFast.from_pretrained(MODEL_BASE)
MASK = tokenizer.mask_token

masked_text = [f"Jim Cramer is consistently bullish when it comes to {MASK}. What this means in practicality is that Cramer routinely recommends buying stocks, and he rarely offers up a sell call. Analysis of his recommendations between 2016 and 2022 (via the data project Jim Cramer's Recommendations: A Six-Year Analysis) shows a 10.32% distribution of {MASK} recommendations alongside 61.27% buys, plus a smattering of positive or negative commentary without a formal buy or sell recommendation attached."]

inputs = tokenizer(masked_text, return_tensors="tf", padding=True, truncation=True)

model = TFBertForMaskedLM.from_pretrained(MODEL_BASE)
logits = model(**inputs).logits
mask_token_idxs = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)
print(mask_token_idxs)
print(logits)

In [ ]:
mask_logits = tf.gather_nd(logits, mask_token_idxs)
top_5 = tf.math.top_k(mask_logits, k=5)
[tokenizer.decode([idx]) for idx in top_5.indices.numpy().flatten()]
for i in range(5):
    new_text = masked_text[0]
    for j in range(2):
        token_idx = top_5.indices[j, i]
        top5_logits = top_5.values[j]

        proba = tf.nn.softmax(top5_logits)
        predicted_token = tokenizer.decode([token_idx])
        new_text = new_text.replace(MASK, f'[{predicted_token}:{proba[i].numpy()*100.:.01f}%]', 1)
    print(new_text)

# Financial Conditioning

In [ ]:
adapt_train_file = os.path.join(DATA_PATH, 'Domain_adapation/train.txt')
adapt_test_file = os.path.join(DATA_PATH, 'Domain_adapation/dev.txt')
def text_dataset(tokenizer, file_path):
    def generator():
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in tqdm(file, desc="text_dataset"):
                tokens = tokenizer(line.strip(),
                                   add_special_tokens=True,
                                   truncation=False,
                                   padding=False)
                yield {
                    'input_ids': tf.ragged.constant([tokens['input_ids']]),
                    'attention_mask': tf.ragged.constant([tokens['attention_mask']])
                }
    return tf.data.Dataset.from_generator(
        generator,
        output_signature={
            'input_ids': tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int32),
            'attention_mask': tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int32)
        })

train_dataset = text_dataset(tokenizer, adapt_train_file)
eval_dataset = text_dataset(tokenizer, adapt_test_file)

iterator = iter(eval_dataset.as_numpy_iterator())
example = next(iterator)
inputs = example['input_ids'][0]
print(f"Input IDs (len: {len(inputs)}):", inputs)
print("Attention Mask:", example['attention_mask'])

The MLM needs chunked sequences which are comprised of the whole corpus concatenated. Chunks are sized on the given hardware or the max dictionary the  tokenizer has - in general 128 is a good number for modern hardward.

As we concatenate, we add a lable column on which the MLM can use as a ground truth

In [ ]:
def chunked_text_dataset(tokenizer, file_path, chunk_len=512):
    all_tokens = []
    all_attention_masks = []
    all_special_tokens_masks = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in tqdm(file, desc="Reading file lines"):
            tokens = tokenizer(line.strip(),
                               truncation=True,
                               add_special_tokens=True,
                               return_special_tokens_mask=True,
                               padding=False)
            all_tokens.extend(tokens['input_ids'])
            all_attention_masks.extend(tokens['attention_mask'])
            all_special_tokens_masks.extend(tokens['special_tokens_mask'])

    def generator():
        num_chunks = len(all_tokens) // chunk_len
        for i in tqdm(range(num_chunks), desc= "chunking...", position=0, leave=True):
            start = i * chunk_len
            end = start + chunk_len
            input_ids_chunk = all_tokens[start:end]
            attention_mask_chunk = all_attention_masks[start:end]
            special_tokens_mask_chunk = all_special_tokens_masks[start:end]

            yield {
                'input_ids': tf.convert_to_tensor(input_ids_chunk, dtype=tf.int32),
                'attention_mask': tf.convert_to_tensor(attention_mask_chunk, dtype=tf.int32),
                'labels': tf.convert_to_tensor(input_ids_chunk, dtype=tf.int32),
                'special_tokens_mask': tf.convert_to_tensor(special_tokens_mask_chunk, dtype=tf.int32)
            }

    return tf.data.Dataset.from_generator(
        generator,
        output_signature={
            'input_ids': tf.TensorSpec(shape=(chunk_len,), dtype=tf.int32),
            'attention_mask': tf.TensorSpec(shape=(chunk_len,), dtype=tf.int32),
            'labels': tf.TensorSpec(shape=(chunk_len,), dtype=tf.int32),
            'special_tokens_mask': tf.TensorSpec(shape=(chunk_len,), dtype=tf.int32)
        })


train_dataset = chunked_text_dataset(tokenizer, adapt_train_file)
iterator = iter(eval_dataset.as_numpy_iterator())
example = next(iterator)
inputs = example['input_ids'][0]
print(f"Input IDs (len: {len(inputs)}):", inputs)
print("Decoded IDs:", tokenizer.decode(inputs))

For MLMs huggingface offers a specific data collector that does the masking. Although we can mask random tokens using the `[MASK]` special token at random intervals, as long as there is a labals column with the ground truth.

In [ ]:
from transformers import DataCollatorForLanguageModeling, BertConfig

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")
batched_dataset = train_dataset.batch(1).take(1)

batch = next(iter(eval_dataset.as_numpy_iterator()))
batch = {k: v for k, v in batch.items()}
examples = [{k: v[i] for k, v in batch.items()} for i in range(batch['input_ids'].shape[0])]
print(examples)
collated_batch = data_collator(examples)
for input_ids, labels in tqdm(zip(collated_batch['input_ids'], collated_batch['labels']), desc="tokenizing batches"):
    masked_text = tokenizer.decode(input_ids)
    original_text = tokenizer.decode([label if label != -100 else input_id for label, input_id in zip(labels, input_ids)])

    print(f"Masked: {masked_text}")
    print(f"Original: {original_text}")

logits = model(**collated_batch)
print(f"logits: {logits}")

Add everything together

In [ ]:
MAX_LEN = 512 # Default 256, MAX 512
def mlm_text_dataset(file_path, tokenizer, data_collator, chunk_len=MAX_LEN):
    all_tokens = []
    all_attention_masks = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in tqdm(file, position=0, leave=True, desc="Processing file..."):
            tokens = tokenizer(line.strip(),
                               truncation=True,
                               add_special_tokens=True,
                               return_special_tokens_mask=True,
                               padding=False)
            all_tokens.extend(tokens['input_ids'])
            all_attention_masks.extend(tokens['attention_mask'])

    num_chunks = len(all_tokens) // chunk_len
    tokens_chunks = []
    attention_mask_chunks = []
    label_chunks = []
    for i in tqdm(range(num_chunks), position=0, leave=True, desc="Chunking..."):
        start = i * chunk_len
        end = start + chunk_len
        input_ids_chunk = all_tokens[start:end]
        attention_mask_chunk = all_attention_masks[start:end]

        masked_chunks = data_collator([{
                'input_ids': tf.convert_to_tensor(input_ids_chunk, dtype=tf.int32),
                'attention_mask': tf.convert_to_tensor(attention_mask_chunk, dtype=tf.int32)}])
        tokens_chunks.extend(masked_chunks['input_ids'])
        label_chunks.extend(masked_chunks['labels'])
        attention_mask_chunks.extend(masked_chunks['attention_mask'])
    return tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': tokens_chunks,
            'attention_mask': attention_mask_chunks,
            'labels': label_chunks
        },
    ))

with strategy.scope():
    tokenizer = BertTokenizerFast.from_pretrained(MODEL_BASE)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="np")
    mlm_train_dataset = mlm_text_dataset(adapt_train_file, tokenizer, data_collator)
    mlm_test_dataset = mlm_text_dataset(adapt_test_file, tokenizer, data_collator)

example = next(iter(mlm_test_dataset.as_numpy_iterator()))
inputs = example[0]['input_ids']
print(f"Input IDs (len: {len(inputs)}):", inputs)
print("Decoded IDs:", tokenizer.decode(inputs))

In [ ]:
BATCH_SIZE = 8 * strategy.num_replicas_in_sync # Default 8
BUFFER_SIZE = 10000

def eval_mlm(model, batched_dataset):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
    total_loss = 0.
    total_accuracy = 0.
    total_examples = 0.

    # TODO: convert this to a TF function for distributed strat.
    for batch in tqdm(batched_dataset, desc="eval_mlm", position=0, leave=True):
        for dataset_output in batch:
            input_ids = dataset_output['input_ids']
            attention_mask = dataset_output['attention_mask']
            labels = dataset_output['labels']

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            mask = (labels != -100)
            masked_logits = tf.boolean_mask(logits, mask)
            masked_labels = tf.boolean_mask(labels, mask)
            batch_loss = loss_fn(masked_labels, masked_logits)
            predictions = tf.argmax(masked_logits, axis=-1)
            batch_accuracy = tf.reduce_sum(tf.cast(tf.equal(predictions, masked_labels), dtype=tf.float32))

            total_loss += tf.cast(batch_loss,tf.float32)
            total_accuracy += batch_accuracy
            total_examples += tf.size(masked_labels, out_type=tf.float32)

    avg_loss = total_loss / total_examples
    avg_perplexity = tf.exp(avg_loss).numpy()
    avg_accuracy = total_accuracy / total_examples

    print(f"Average Cross-Entropy Loss: {avg_loss.numpy()}")
    print(f"Average Perplexity: {avg_perplexity}")
    print(f"Average Accuracy: {avg_accuracy.numpy()}")

config = BertConfig.from_pretrained(MODEL_BASE)
model = TFBertForMaskedLM.from_pretrained(MODEL_BASE, config=config)
test_dataset = mlm_test_dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)
eval_mlm(model, test_dataset)

In [ ]:
MAX_LEN = 512 # Default 256, MAX 512
LEARN_RATE=5e-5 # 5e-5
LR_FACTOR=0.1
LR_MINDELTA=1e-4
EPOCHS=30
PATIENCE=10

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import AdamW

with strategy.scope():
    config = BertConfig.from_pretrained(MODEL_BASE)
    cond_model = TFBertForMaskedLM.from_pretrained(MODEL_BASE, config=config)
    
    # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
    tensorboard_callback = TensorBoard(log_dir=f"{MODEL_PATH}/logs",
                                        histogram_freq=2,
                                        embeddings_freq=2)
    # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    early_stopping = EarlyStopping(mode='min', patience=PATIENCE, start_from_epoch=1)
    #tf.debugging.enable_check_numerics() # - Assert if no Infs or NaNs go through. not for TPU!
    #tf.config.run_functions_eagerly(not is_tpu_strategy(strategy)) # - Easy debugging
    # https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
    train_dataset = (mlm_train_dataset.shuffle(buffer_size=BUFFER_SIZE)
                                    .batch(BATCH_SIZE)
                                    .cache()
                                    .prefetch(tf.data.experimental.AUTOTUNE))
    test_dataset = (mlm_test_dataset.shuffle(buffer_size=BUFFER_SIZE)
                                    .batch(BATCH_SIZE)
                                    .cache()
                                    .prefetch(tf.data.experimental.AUTOTUNE))
    cond_model.compile(optimizer=AdamW(learning_rate=LEARN_RATE))
    history = cond_model.fit(train_dataset,
                        epochs=EPOCHS,
                        callbacks=[tensorboard_callback, early_stopping],
                        verbose="auto",
                        validation_data=test_dataset)

In [ ]:
import zipfile
def zip_models(directory, output_filename):
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(directory, '..')))


cond_model.save_pretrained(f"{MODEL_PATH}/model")
config.save_pretrained(f"{MODEL_PATH}/config")
tokenizer.save_pretrained(f"{MODEL_PATH}/tokenizer")

zip_models(MODEL_PATH, './cond_bert.zip')

# Entropy

Entropy is a measure that quantifies uncertainty or the inverse of probability of an event occurring;h igher the probability, lesser is the uncertainty. 


Hence, the goal of the language model is to minimize the entropy of generating a sequence of words that are similar to the training sequences. The formula for calculating Entropy is as given below where P(x) is the probability of the word x:

$$
H(X) = -\sum_{i=1}^n P(x_i) \log_b P(x_i)
$$

Where:
* H(X) is the entropy of the random variable X, which represents the different outcomes in the language model
* P(x_i) is the probability of occurrence of each outcome x_i
* n is the number of possible outcomes.
* logb is the logarithm base, e.g base 2 for binary entropy calculations.

Cross enthropy measures 2 distributions the true outcome distributions and the models. Using the equaltion above the second p(x_i) is replaced with the models distribution.

# Perplexity

Perplexity means the model is surprised to see new data. The lower the perplexity, the better the training is.

The formula for perplexity is the exponent of mean of log likelihood of all the words in an input sequence:

$$
\text{PPL}(X) = \exp\left(-\frac{1}{T} \sum_{i=1}^T \log p_{\theta}(x_i | x_{< i}) \right)
$$

In [ ]:
train_loss = history.history["loss"][-1]
try:
    train_perplexity = math.exp(train_loss)
except OverflowError:
    train_perplexity = math.inf
validation_loss = history.history["val_loss"][-1]
try:
    validation_perplexity = math.exp(validation_loss)
except OverflowError:
    validation_perplexity = math.inf
results_dict = {}
results_dict["train_loss"] = train_loss
results_dict["train_perplexity"] = train_perplexity
results_dict["eval_loss"] = validation_loss
results_dict["eval_perplexity"] = validation_perplexity

results_dict

# Evaluate Conditioned



In [ ]:
config = BertConfig.from_pretrained(f"./{MODEL_PATH}/config")
model = TFBertForMaskedLM.from_pretrained(f"./{MODEL_PATH}/model", config=config)
test_dataset = mlm_test_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

eval_mlm(model, test_dataset)